In [1]:
# Computing and applying a linear minimum-norm inverse method on evoked/raw/epochs data

# Import necessary libraries and functions
from scipy.stats import zscore
import sklearn as sk
from scipy import signal
import copy  # This is a Python module that allows you to copy objects without changing the original object
from mne.preprocessing import ICA
from mne_icalabel import label_components
from sklearn.decomposition import PCA
from mne.minimum_norm import apply_inverse_epochs
from mne.minimum_norm import make_inverse_operator, apply_inverse
from mne.datasets import sample, eegbci, fetch_fsaverage
import seaborn as sns  # required for heatmap visualization
import networkx as nx
from scipy.stats import pearsonr
from mne.viz import circular_layout
from mne_connectivity.viz import plot_connectivity_circle
from scipy.signal import hilbert
import mne
import matplotlib.pyplot as plt
import os
from mne.datasets import fetch_fsaverage
from nilearn import datasets
import os.path as op
import numpy as np
import matplotlib
# matplotlib.use('Qt5Agg')  # Setting the backend BEFORE importing pyplot

# mne.viz.set_3d_backend("pyvista")


#################################################################################
# Adult template MRI (fsaverage)

# Download fsaverage files
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = op.dirname(fs_dir)

# The files live in:
subject = "fsaverage"
trans = "fsaverage"  # MNE has a built-in fsaverage transformation
src = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
bem = op.join(fs_dir, "bem", "fsaverage-5120-5120-5120-bem-sol.fif")


# Import necessary Python modules
matplotlib.use('Agg')  # disable plotting
mne.viz.set_browser_backend('matplotlib', verbose=None)
mne.set_config('MNE_BROWSER_BACKEND', 'matplotlib')


# defining input and output directory
files_in = '../data/in/subjects/'
files_out = '../data/out/subjects/'


# loading list of subject names from txt file
# names = open("./names.txt", "r")
names = []

subject_list = names.read().split('\n')
modes = ['EC', 'EO']
# Read the custom montage
montage_path = r"../data/in/MFPRL_UPDATED_V2.sfp"
montage = mne.channels.read_custom_montage(montage_path)

# Define the map of channel names using the provided keys
ch_map = {'Ch1': 'Fp1', 'Ch2': 'Fz', 'Ch3': 'F3', 'Ch4': 'F7', 'Ch5': 'LHEye', 'Ch6': 'FC5',
          # Setting FPz as GND so it matches montage
          'Ch7': 'FC1', 'Ch8': 'C3', 'Ch9': 'T7', 'Ch10': 'GND', 'Ch11': 'CP5', 'Ch12': 'CP1',
          'Ch13': 'Pz', 'Ch14': 'P3', 'Ch15': 'P7', 'Ch16': 'O1', 'Ch17': 'Oz', 'Ch18': 'O2',
          'Ch19': 'P4', 'Ch20': 'P8', 'Ch21': 'Rmastoid', 'Ch22': 'CP6', 'Ch23': 'CP2', 'Ch24': 'Cz',
          'Ch25': 'C4', 'Ch26': 'T8', 'Ch27': 'RHEye', 'Ch28': 'FC6', 'Ch29': 'FC2', 'Ch30': 'F4',
          'Ch31': 'F8', 'Ch32': 'Fp2', 'Ch33': 'AF7', 'Ch34': 'AF3', 'Ch35': 'AFz', 'Ch36': 'F1',
          'Ch37': 'F5', 'Ch38': 'FT7', 'Ch39': 'FC3', 'Ch40': 'FCz', 'Ch41': 'C1', 'Ch42': 'C5',
          'Ch43': 'TP7', 'Ch44': 'CP3', 'Ch45': 'P1', 'Ch46': 'P5', 'Ch47': 'Lneck', 'Ch48': 'PO3',
          'Ch49': 'POz', 'Ch50': 'PO4', 'Ch51': 'Rneck', 'Ch52': 'P6', 'Ch53': 'P2', 'Ch54': 'CPz',
          'Ch55': 'CP4', 'Ch56': 'TP8', 'Ch57': 'C6', 'Ch58': 'C2', 'Ch59': 'FC4', 'Ch60': 'FT8',
          'Ch61': 'F6', 'Ch62': 'F2', 'Ch63': 'AF4', 'Ch64': 'RVEye'}

for subject in subject_list:
    for mode in modes:
        print(subject, mode)
        # defining paths for current subject
        input_path = files_in+subject + '/' + mode + '/'
        output_path = files_out + subject + '/' + mode + '/'
        stc_path  = output_path +'stc/'
        
        schaefer_atlas = datasets.fetch_atlas_schaefer_2018(n_rois=100)
        src = mne.read_source_spaces(
            files_in + '../fsaverage/bem/fsaverage-ico-5-src.fif', patch_stats=False, verbose=None)


        # Load inverse solution file paths for both left and right hemispheres
        inverse_solution_files_lh = []
        inverse_solution_files_rh = []

        for path, subdirs, files in os.walk(stc_path):
            for file in files:
                filepath = path + file
                if '-rh.stc' in file:
                    inverse_solution_files_rh.append(filepath)
                elif '-lh.stc' in file:
                    inverse_solution_files_lh.append(filepath)

        

0 files missing from root.txt in /home/prakhar/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/prakhar/mne_data/MNE-fsaverage-data/fsaverage
Using matplotlib as 2D backend.


FileNotFoundError: [Errno 2] No such file or directory: './names.txt'

In [6]:

for path, subdirs, files in os.walk('./'):
    for file in files:
        print(path + file)

./src_parcel.ipynb
./EEG_src_parcel.py
./EEG_source_localization_2D.py
./EEG_fc_global_graph_theoretical_disparity_filter.py
./EEG_preprocessing_v2.py
